In [31]:
# Utility Functions

In [32]:
import numpy as np
import matplotlib.pyplot as plt

In [33]:
def get_music_notes():
    '''
    Get the frequency in hertz for all keys on a standard saxophone.

    Returns
    -------
    note_freqs : dict
        Mapping between note name and corresponding frequency.

    '''
    
    # Major keys in capital letter and 
    octave = ['C', 'c', 'D', 'd', 'E', 'F', 'f', 'G', 'g', 'A', 'a', 'B'] 
    base_freq = 440 #Frequency of Note A4
    keys = np.array([x+str(y) for y in range(0,9) for x in octave])
    # Trim to standard 88 scale
    start = np.where(keys == 'A0')[0][0]
    end = np.where(keys == 'C8')[0][0]
    keys = keys[start:end+1]
    
    note_freqs = dict(zip(keys, [2**((n+1-49)/12)*base_freq for n in range(len(keys))]))
    note_freqs[''] = 0.0 # stop
    return note_freqs

In [34]:
def get_sine_wave(frequency, duration, sample_rate=44100, amplitude=4096):
    '''
    Get pure sine wave. 

    Parameters
    ----------
    frequency : float
        Frequency in hertz.
    duration : float
        Time in seconds.
    sample_rate : int, optional
        Wav file sample rate. The default is 44100.
    amplitude : int, optional
        Peak Amplitude. The default is 4096.

    Returns
    -------
    wave : TYPE
        DESCRIPTION.

    '''
    t = np.linspace(0, duration, int(sample_rate*duration))
    wave = amplitude*np.sin(2*np.pi*frequency*t)
    return wave

## A few comments on Fourier Series here please

In [35]:
def fourierSeriesCoefficients(period, N):
    '''
    Generate the fourier series coefficients: an, bn

    Parameters
    ----------
    period : float
        Period of the wave
    N : int
        The harmonic to generate the series up to
    '''

    result = []

    T = len(period)
    t = np.arange(T)

    for n in range(N+1):
        an = 2/T * (period * np.cos(2*np.pi*n*t/T)).sum()
        bn = 2/T * (period * np.sin(2*np.pi*n*t/T)).sum()
        result.append((an, bn))
    
    return np.array(result)

In [36]:
def extractPeriod(data, rate, t_start, t_end):
  
    sample_start = int(t_start * rate)
    sample_end = int(t_end * rate)

    period = data[sample_start:sample_end]
    
    return period, rate

In [37]:
def translate(T, coeffs):
    '''
    Sum up the sine and cosine of 

    Parameters
    ----------
    T : int
        Number of samples in one period
    coeffs : NumPy array
        NumPy array of the fourier coefficients
    '''

    result = 0

    t = np.arange(T)
    for n, (an, bn) in enumerate(coeffs):
        if n  == 0:
            an = an/2
        result = result + an*np.cos(2*np.pi*n*t/T) + bn*np.sin(2*np.pi*n*t/T)
    
    return result

In [38]:
def apply_saxophone_overtones(frequency, duration, sax_factor, sample_rate=44100, amplitude=4096):
    
    # assert sum(sax_factor) <= 1  
    
    frequencies = np.minimum(np.array([frequency * (x - 5) for x in range(len(sax_factor))]), sample_rate // 2)
    amplitudes = np.array([amplitude * (x * 2.5) for x in sax_factor])
    
    fundamental = get_sine_wave(frequencies[0], duration, sample_rate, amplitudes[0])
    for i in range(1, len(sax_factor)):
        overtone = get_sine_wave(frequencies[i], duration, sample_rate, amplitudes[i])
        fundamental += overtone
    
    return fundamental


In [39]:
import numpy as np

def apply_adsr_envelope(input_wave, sample_rate, attack_time, decay_time, sustain_level, release_time):
    """
    Apply an ADSR envelope to the input wave.

    Args:
    - input_wave (numpy array): The wave to modulate.
    - sample_rate (int): The sample rate of the audio signal.
    - attack_time (float): Time for the attack phase in seconds.
    - decay_time (float): Time for the decay phase in seconds.
    - sustain_level (float): Amplitude level during the sustain phase, relative to peak.
    - release_time (float): Time for the release phase in seconds.

    Returns:
    - numpy array: The wave modulated by the ADSR envelope.
    """
    num_samples = len(input_wave)
    attack_samples = int(sample_rate * attack_time)
    decay_samples = int(sample_rate * decay_time)
    release_samples = int(sample_rate * release_time)

    # ADSR Envelope Construction
    envelope = np.zeros(num_samples)
    
    # Attack
    if attack_samples > 0:
        envelope[:attack_samples] = np.linspace(0, 1, attack_samples)
    
    # Decay
    decay_start = attack_samples
    decay_end = decay_start + decay_samples
    if decay_samples > 0:
        envelope[decay_start:decay_end] = np.linspace(1, sustain_level, decay_samples)
    
    # Sustain
    sustain_start = decay_end
    sustain_end = num_samples - release_samples
    envelope[sustain_start:sustain_end] = sustain_level
    
    # Release
    if release_samples > 0:
        envelope[sustain_end:] = np.linspace(sustain_level, 0, release_samples)
    
    # Apply envelope to the input wave
    modulated_wave = input_wave * envelope

    return modulated_wave


In [40]:
from scipy.signal import butter, filtfilt

def compute_envelope_lowpass(signal, sample_rate, cutoff_freq=10, order=4):
    """
    Compute the amplitude envelope of a signal using low-pass filtering.

    Args:
    - signal (numpy array): The input audio signal.
    - sample_rate (int): The sample rate of the audio signal.
    - cutoff_freq (float): Cutoff frequency for the low-pass filter.
    - order (int): Order of the filter.

    Returns:
    - numpy array: The smoothed amplitude envelope of the signal.
    """
    b, a = butter(order, cutoff_freq / (0.5 * sample_rate), btype='low')
    envelope = filtfilt(b, a, np.abs(signal))  # Apply the filter to the rectified signal
    return envelope


In [41]:
def apply_amplitude_envelope(input_wave, envelope):
    """
    Apply an amplitude envelope to the input wave.

    Args:
    - input_wave (numpy array): The wave to modulate.
    - envelope (numpy array): The amplitude envelope array.

    Returns:
    - numpy array: The amplitude-modulated wave.
    """
    # Ensure the envelope is the same length as the input wave
    if len(input_wave) != len(envelope):
        raise ValueError("The input wave and envelope must be the same length.")

    # Apply the envelope to the wave
    modulated_wave = input_wave * envelope
    
    return modulated_wave


In [42]:
def compute_envelope_hilbert(signal):
    """
    Compute the amplitude envelope of a signal using the Hilbert transform.

    Args:
    - signal (numpy array): The input audio signal.

    Returns:
    - numpy array: The amplitude envelope of the signal.
    """
    analytic_signal = hilbert(signal)  # Compute the analytic signal
    envelope = np.abs(analytic_signal)  # The envelope is the magnitude of the analytic signal
    
    return envelope